In [ ]:
!git clone https://github.com/PedroOrtix/ID_Inpainter.git
%cd ID_Inpainter
!pip install -r requirements.txt -q
!pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html -q

In [ ]:
%cd ID_Inpainter

In [ ]:
from paddleocr import PaddleOCR
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
from scipy import ndimage

from src.text_inpaint.utils import separar_cadenas, mostrar_diccionario_ascii

import src.text_inpaint.ocr as ocr
from src.text_inpaint.ocr import recalcular_cuadricula_rotada
from src.text_inpaint.ocr import comparar_imagenes
from src.text_inpaint.ocr import reemplazar_parte_imagen
from src.text_inpaint.ocr import juntar_imagenes_vertical, rellenar_imagen_uniformemente
from src.text_inpaint.ocr import recortar_imagen_uniformemente
from src.text_inpaint.run import simple_inpaint



In [ ]:
#open a file containing the image
img = Image.open('images/imagen_dni.jpg')

# SOLO SI LA IMAGEN TIENE MUCHA RESOLUCIÓN
# Especificar el nuevo tamaño (por ejemplo, la mitad de la resolución original)
# nuevo_tamaño = (img.width // 2, img.height // 2)
# # Redimensionar la imagen
# img = img.resize(nuevo_tamaño, Image.ANTIALIAS)

img_array = np.array(img)

# Inicializar el modelo OCR Paddle
model = PaddleOCR(use_angle_cls=True, lang='es')  # 'es' para español
# Realiza OCR
result = model.ocr(img_array)
bounds = ocr.convert_paddle_to_easyocr(result)

# Imprime los nombres (textos) de los cuadros delimitadores
# tres nombres /n tres nombres
lista_elementos_detectados = []
for bound in bounds:
    lista_elementos_detectados.append(bound[1])

dict_elems = separar_cadenas(lista_elementos_detectados)
mostrar_diccionario_ascii(dict_elems)

In [ ]:
palabra = "039936"
replace = "127648"
slider_step = 30
slider_guidance = 2
slider_batch = 6

In [ ]:
from src.text_inpaint.run import process_image
modified_images, right_bounds, coordenadas_originales = process_image(palabra = palabra,
                replace = replace,
                bounds=bounds,
                img_array=img_array,
                height=256,
                weight=512,
                slider_step=30,
                slider_guidance=2,
                slider_batch=6,
                show_plot=True,
                save_intermediate_images=True)


In [ ]:
# elegir cual te gusta mas
modified_images[5].save("images/modified_image.jpg")

In [ ]:
# ver la comparacion entre lo que se ha hecho inpaint
coodinates = np.array(right_bounds[0][0], dtype = int).flatten()
comparar_imagenes("images/imagen_dni_recortada.jpg",
                  "images/modified_image.jpg",
                  coodinates,
                  save_path="./",
                  adjust_temp=False)


In [ ]:
img_pil = Image.open("images/imagen_dni_recortada.jpg")

In [ ]:
# reemplazar la parte de la imagen original(512x512) por la modificada (boundong box modificada)
img_recortada_mod = reemplazar_parte_imagen(img_pil,
                                            modified_images[5],
                                            coodinates,
                                            adjust_temp=False)

In [ ]:
img_recortada_mod.save("images/img_recortada_mod.jpg")

In [ ]:
#return to the originar size before the padding
img_recortada_mod, _ = recortar_imagen_uniformemente(img_recortada_mod)
# take the half of the image horizontally of img_recortad_mod
img_recortada_mod = img_recortada_mod.crop((0, 0, img_recortada_mod.width, img_recortada_mod.height//2))
img_recortada_mod.save("images/img_recortada_mod.jpg")

In [ ]:
# superponer la imagen 512x512 modificada a la original
img_array = img_array[:, :, :3]
x_min, y_min, x_max, y_max = coordenadas_originales
img_array[y_min:y_max, x_min:x_max] = np.array(img_recortada_mod)[:, :, :]

In [ ]:
Image.fromarray(img_array).save("images/resultado_final.jpg")